In [3]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [4]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools

In [5]:
data_path = "/home/denis/workspace/data/ivt_fields_v1"

function get_files_of_member(scenario_id, member_nr)
    return readdir(joinpath(data_path, scenario_id, "r$(member_nr)i1p1f1"), join = true)    
end


get_files_of_member (generic function with 1 method)

In [6]:
function get_eof_of_datachunk(data)
    eof = EmpiricalOrthogonalFunction(data; timedim=3)

    temporalsignal = pcs(eof)
    spatialsignal = reshape(eofs(eof),(size(datain)[1:2]..., :))
    return temporalsignal, spatialsignal
end

get_eof_of_datachunk (generic function with 1 method)

In [9]:
function get_all_data(scenario_id, member_nr)

    file_paths = get_files_of_member(scenario_id, member_nr)

    time_data = Union{Missing, Dates.DateTime}[]
    ivt_data = Array{Union{Missing, Float64}, 3}[]

    dims_first = NCDataset(file_paths[1]) do ds
        return size(ds[:ivt])
    end

    for file_path in file_paths
        time_chunk, ivt_chunk = NCDataset(file_path) do ds
            return ds[:time][:]::Array{Union{Missing, Dates.DateTime}, 1}, ds[:ivt][:, :, :]::Array{Union{Missing, Float64}, 3}
        end
        append!(time_data, time_chunk)
        push!(ivt_data, ivt_chunk)
    end

    return time_data, cat(ivt_data..., dims=3)
end

get_all_data (generic function with 1 method)

In [10]:
time_data, ivt_data = get_all_data("ssp126", 10)

(Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2100-12-29T18:00:00"), DateTime("2100-12-30T00:00:00"), DateTime("2100-12-30T06:00:00"), DateTime("2100-12-30T12:00:00"), DateTime("2100-12-30T18:00:00"), DateTime("2100-12-31T00:00:00"), DateTime("2100-12-31T06:00:00"), DateTime("2100-12-31T12:00:00"), DateTime("2100-12-31T18:00:00"), DateTime("2101-01-01T00:00:00")], Union{Missing, Float64}[107.83024404217494 105.35183652372383 … 4.810704767513298 14.481626430503018; 138.11806601868335 132.92639636951492 … 5.865105532526301 14.261222253533777; … ; 32.42546716028079 93.37890870191428 … 7.1671847647811555 15.350266308663777; 35.96681207349521 100.77858958675463 … 6.00310975

In [11]:
size(ivt_data)

(70, 32, 125644)

In [13]:
function filter_by_date(fun, time_data, ivt_data, time_dim = 3)

    time_result = filter(fun, time_data)
    data_result = [ivt_data[:, :, i] for i in eachindex(time_data) if fun(time_data[i])]

    return time_result, cat(data_result..., dims=3)
end

filter_by_date (generic function with 2 methods)

In [14]:
time_filtered, data_filtered = filter_by_date(time_data, ivt_data) do time_element
    winter_months = [11, 12, 1, 2, 3]

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

In [1]:
res = cat(data_filtered..., dims=3)

UndefVarError: UndefVarError: `data_filtered` not defined